In [1]:
import torch
import json
import pandas as pd
import numpy as np
from datasets import load_dataset
from FlagEmbedding import BGEM3FlagModel
from FlagEmbedding import AbsEmbedder
from transformers import AutoModel

### 01 模型结构的修改的对比

In [2]:
### 训练后的模型的path
path_bef="./huggingface/hub/models--BAAI--bge-m3/snapshots/5617a9f61b028005a4858fdac845db406aefb181"
path_end="./model/BGEm3_Law_new/"

model_bef = AutoModel.from_pretrained(path_bef)
print(sum(param.numel() for name,param in model_bef.named_parameters()))

model_end = AutoModel.from_pretrained(path_end)
print(sum(param.numel() for name,param in model_end.named_parameters()))


# 检查两个模型是否真的是不同的实例
print(f"模型相同: {model_bef is model_end}")

# 检查模型参数是否相同
for (name1, param1), (name2, param2) in zip(model_bef.named_parameters(), model_end.named_parameters()):
    print(f"模型名称：{name1,name2}")
    print(f"参数的值：{param1,param2}")
    ## 判断参数数目是否一致：
    if torch.allclose(param1,param2):
        # 使用默认的绝对误差容限（atol）和相对误差容限（rtol）
        print("参数的值一致")
    else:
        print("参数的值不同")

567754752
567754752
模型相同: False
模型名称：('embeddings.word_embeddings.weight', 'embeddings.word_embeddings.weight')
参数的值：(Parameter containing:
tensor([[-0.0757, -0.0167, -0.0790,  ...,  0.0348,  0.0125, -0.0478],
        [-0.0029,  0.0035,  0.0062,  ..., -0.0028,  0.0084, -0.0044],
        [-0.0415,  0.0035, -0.0773,  ...,  0.0975,  0.1038,  0.1824],
        ...,
        [ 0.1523,  0.2382,  0.2328,  ...,  0.0466,  0.1892, -0.1609],
        [ 0.0424, -0.0289, -0.1057,  ...,  0.0868,  0.1027,  0.0886],
        [ 0.0360,  0.0206, -0.0455,  ...,  0.0236,  0.0833,  0.0753]],
       requires_grad=True), Parameter containing:
tensor([[-0.0757, -0.0167, -0.0790,  ...,  0.0348,  0.0125, -0.0478],
        [-0.0029,  0.0035,  0.0062,  ..., -0.0028,  0.0084, -0.0044],
        [-0.0415,  0.0035, -0.0773,  ...,  0.0975,  0.1038,  0.1824],
        ...,
        [ 0.1523,  0.2382,  0.2328,  ...,  0.0466,  0.1892, -0.1609],
        [ 0.0424, -0.0289, -0.1057,  ...,  0.0868,  0.1027,  0.0886],
        [ 0.0

### 02 微调后模型效果的评估

#### 1、 模型及向量数据库加载

In [3]:
#### 2、向量数据库加载
path="./model/BGEm3_Law_new/"
model = BGEM3FlagModel(path,pooling_method='cls',use_fp16=False,return_sparse = True,trust_remote_code=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [4]:
### 2.1、数据加载 + 批量向量化
import pandas as pd
data = pd.read_csv("./law_faq.csv")
data=data.dropna()

In [5]:
query_list=list(data["title"])
reply=list(data["reply"])
knowledge_embedder=model.encode(reply)

from faiss_index import Faissindex
#  采用暴力检索的方式，即计算查询向量与所有数据库向量之间的距离，然后返回相似度最高的前k个向量。
#  归一化vectors
vectors=knowledge_embedder['dense_vecs']

sl_fassi=Faissindex(1024)
sl_fassi.create(vectors)

pre tokenize: 100%|██████████| 72/72 [00:02<00:00, 27.32it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 72/72 [01:08<00:00,  1.05it/s]


In [6]:
### 检索的例子
### 对于i的结果进行测试：
i=1
print(query_list[i])
print(reply[i])

vectors_single_test=np.reshape(model.encode(query_list[i]).get('dense_vecs'), (1, -1))
scores_test,index_test=sl_fassi.search(vectors_single_test,5)

print(index_test)
topk_result = data["reply"].values[index_test[0].tolist()]
topk_result

盗窃罪的犯罪客体是什么，盗窃罪的犯罪主体
盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般是指动产而言，但不动产上之附着物，可与不动产分离的，例如，田地上的农作物，山上的树木、建筑物上之门窗等，也可以成为本罪的对象。另外，能源如电力、煤气也可成为本罪的对象。盗窃罪侵犯的客体是公私财物的所有权。所有权包括占有、使用、收益、处分等权能。这里的所有权一般指合法的所有权，但有时也有例外情况。根据《最高人民法院关于审理盗窃案件具体应用法律若干问题的解释》(以下简称《解释》)的规定：“盗窃违禁品，按盗窃罪处理的，不计数额，根据情节轻重量刑。盗窃违禁品或犯罪分子不法占有的财物也构成盗窃罪。”
[[    1 10688 10414 10780 12320]]


array(['盗窃罪的客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般是指动产而言，但不动产上之附着物，可与不动产分离的，例如，田地上的农作物，山上的树木、建筑物上之门窗等，也可以成为本罪的对象。另外，能源如电力、煤气也可成为本罪的对象。盗窃罪侵犯的客体是公私财物的所有权。所有权包括占有、使用、收益、处分等权能。这里的所有权一般指合法的所有权，但有时也有例外情况。根据《最高人民法院关于审理盗窃案件具体应用法律若干问题的解释》(以下简称《解释》)的规定：“盗窃违禁品，按盗窃罪处理的，不计数额，根据情节轻重量刑。盗窃违禁品或犯罪分子不法占有的财物也构成盗窃罪。”',
       '根据刑法第264条的规定，盗窃罪是指以非法占有为目的，窃取公私财物数额较大或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃公私财物的行为。构成要件客体要件本罪侵犯的客体是公私财物的所有权。侵犯的对象，是国家、集体或个人的财物，一般是指动产而言，但不动产上之附着物，可与不动产分离的，例如，田地上的农作物，山上的树木、建筑物上之门窗等，也可以成为本罪的对象。另外，能源如电力、煤气也可成为本罪的对象。盗窃罪侵犯的客体是公私财物的所有权。所有权包括占有、使用、收益、处分等权能。这里的所有权一般指合法的所有权，但有时也有例外情况。根据《最高人民法院关于审理盗窃案件具体应用法律若干问题的解释》（以下简称《解释》的规定：“盗窃违禁品，按盗窃罪处理的，不计数额，根据情节轻重量刑。盗窃违禁品或犯罪分子不法占有的财物也构成盗窃罪。”客观要件本罪在客观方面表现为行为人具有窃取数额较大的公私财物或者多次窃取公私财物、入户盗窃、携带凶器盗窃、扒窃的行为。所谓窃取，是指行为人违反被害人的意志，将他人占有的财物转移为自己或第三者（包括单位）占有。主体要件本罪主体是一般主体，凡达到刑事责任年龄（16周岁）且具备刑事责任能力的人均能构成。对主体的修改是对本罪修改的重要内容。依原刑法，已满14岁不满16岁的少年犯惯窃罪、重大盗窃罪的，应当负刑事责任。本法取消了此规定。主观要件本罪在主观方面表现为直接故意，且具有非法占有的目的。',
       '根据刑法第264条的规定，盗窃罪是指以非法占有为目的，窃取公私财物数额较大或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃公私财物的行为。构成要件：客体要件

#### 2、计算原模型的前30位的计算错误比例

In [7]:
### 计算结果：
from tqdm import tqdm
bench_select=[10,20,30]

for bench in  bench_select:
    wrong_index=[]
    
    for i, query_x in tqdm(enumerate(query_list), desc="Processing queries"):
        ## 进行检索：
        vectors_single=model.encode(query_x).get('dense_vecs')
        vectors_single=np.reshape(vectors_single, (1, -1))
        scores, indexes=sl_fassi.search(vectors_single,bench)
        is_corr=0
        for index in indexes[0]:
            if bool(index==i):
                is_corr=is_corr+1
        if is_corr==0:
            wrong_index.append(i)
            
    print(f"Bench为{bench}的情况下，错误比例为：{len(wrong_index)/len(query_list)*100}%")

Processing queries: 18208it [05:17, 57.27it/s]


Bench为10的情况下，错误比例为：49.90114235500879%


Processing queries: 18208it [05:11, 58.38it/s]


Bench为20的情况下，错误比例为：42.04195957820738%


Processing queries: 18208it [05:10, 58.56it/s]

Bench为30的情况下，错误比例为：37.59885764499121%
